# SFI 2024 - Rozwiązując korki
### Notebook 3 - Analiza centralności

Finalnie zajmiemy się analizą miar centralności dla Krakowa i porównamy wyniki z typową sytuacją na ulicach miasta.

In [2]:
import osmnx as ox
import networkx as nx
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.colors as mcolor

import pickle
import folium
import matplotlib

from matplotlib import cm

## Funkcje pomocniczne

In [3]:
def engineer_features(G: nx.MultiDiGraph) -> nx.MultiDiGraph:
    for u, v, a in G.edges(data=True):
        if 'maxspeed' not in a:
            a['maxspeed'] = 50
        if isinstance(a['maxspeed'], list):
            a['maxspeed'] = min(a['maxspeed'])
        elif isinstance(a['maxspeed'], dict):
            print(a)

        if isinstance(a['maxspeed'], str):
            a['maxspeed'] = float(a['maxspeed'])

        a['travel_time'] = a['length'] / a['maxspeed']

        G[u][v][0]['travel_time'] = a['travel_time']
    return G


def find_roads(g: nx.MultiDiGraph, road_name: str) -> nx.MultiDiGraph:
    names = set()
    for u, v, a in g.edges(data=True):
        if 'name' in a and road_name in a['name']:
            names.add(a['name'])
    return names


def delete_road(g: nx.MultiDiGraph, road_name: str, inplace: bool = False):
    if not inplace:
        g = g.copy()

    edges_to_remove = []
    for u, v, a in g.edges(data=True):
        if ('name' in a and road_name == a['name']) or ('ref' in a and road_name == a['ref']):
            edges_to_remove.append((u, v))
    for u, v in edges_to_remove:
        g.remove_edge(u, v)
    print(f'Deleted {len(edges_to_remove)} edges')
    return g


def read_graph(city: str, tolerance: float, buffer: float = None):
    """
    
    :param city: City (or location) name 
    :param tolerance: Intersection consolidation tolerance radius in meters
    :param buffer: Graph area buffer in meters
    :return: 
    """
    G = ox.graph_from_place(city,
                            network_type='drive',
                            buffer_dist=buffer)
    G = ox.project_graph(G)
    G = ox.consolidate_intersections(G, tolerance=tolerance)
    return engineer_features(G)


In [11]:
def get_min_max_prop(_streets, attr):
    p = _streets[attr]
    return p.min(), p.max()


def get_cmap(vmin, vmax, cmap='plasma'):
    cmap = cm.get_cmap(cmap)
    norm = mcolor.Normalize(vmin=vmin, vmax=vmax)
    final_cmap = cm.ScalarMappable(norm=norm, cmap=cmap)
    return final_cmap


def edge_embedding(hex_val):
    return {'color': hex_val, 'weight': '3'}


def node_embedding(hex_val):
    return {'color': hex_val}


def style_fun(properties, embed, attr='maxspeed', vmin=0, vmax=100, *_args):
    properties = properties['properties']
    final_cmap = get_cmap(vmin, vmax)

    if attr in properties and properties[attr] is not None:
        if isinstance(properties[attr], list):
            speed = min(map(float, properties[attr]))
        else:
            speed = float(properties[attr])
        rgba = final_cmap.to_rgba(speed)
        hex_val = mcolor.rgb2hex(rgba)
    else:
        hex_val = '#000000'

    return embed(hex_val)


def edges_geojson(_streets, attr):
    vmin, vmax = get_min_max_prop(_streets, attr)
    return folium.GeoJson(_streets,
                          style_function=lambda *x: style_fun(embed=edge_embedding, vmin=vmin, vmax=vmax, attr=attr, *x))


def nodes_geojson(_nodes, attr):
    vmin, vmax = get_min_max_prop(_nodes, attr)
    return folium.GeoJson(_nodes,
                          marker=folium.CircleMarker(
                              radius=3,
                          ),
                          style_function=lambda *x: style_fun(embed=node_embedding, vmin=vmin, vmax=vmax, attr=attr, *x)
                          )

def generate_folium(G, city_name, edge_attr='maxspeed', node_attr=None):
    nodes, streets = ox.graph_to_gdfs(G)
    m = folium.Map(location=ox.geocode(city_name))
    edges_geojson(streets, edge_attr).add_to(m)
    if node_attr is not None:
        nodes_geojson(nodes, node_attr).add_to(m)
    return m


In [5]:
G = read_graph('Kraków', tolerance=20, buffer=0)

# generate_folium(G, 'Kraków')

/tmp/ipykernel_13745/1393963325.py:49: FutureWarning: The buffer_dist argument has been deprecated and will be removed in the v2.0.0 release. Buffer your query area directly, if desired.
  G = ox.graph_from_place(city,
/home/thmtt/miniconda3/envs/sfi_2024/lib/python3.10/site-packages/osmnx/graph.py:379: FutureWarning: The buffer_dist argument has been deprecated and will be removed in the v2.0.0 release. Buffer your results directly, if desired.
  gdf_place = geocoder.geocode_to_gdf(


In [6]:
G = read_graph('Kraków', tolerance=20, buffer=5_000)

# generate_folium(G, 'Kraków')

/tmp/ipykernel_13745/1393963325.py:49: FutureWarning: The buffer_dist argument has been deprecated and will be removed in the v2.0.0 release. Buffer your query area directly, if desired.
  G = ox.graph_from_place(city,
/home/thmtt/miniconda3/envs/sfi_2024/lib/python3.10/site-packages/osmnx/graph.py:379: FutureWarning: The buffer_dist argument has been deprecated and will be removed in the v2.0.0 release. Buffer your results directly, if desired.
  gdf_place = geocoder.geocode_to_gdf(


### W ramach grafu wczytujemy znacznie większy obszar niż sam Kraków. Zastanów się dlaczego?

Podpowiedź:
Jak obszary ościenne wpływają na metryki centralności?

## Na podstawie pliku base_centrality.pickle (output funkcji betweenness centrality i edge betweenness) wstaw atrybut centralności do grafu G.

In [7]:
with open('base_centrality.pickle', 'rb') as f:
    nodes_cen, edges_cen = pickle.load(f)

## Korzystając z funkcji generate_folium, narysuj graf Krakowa z centralnością krawędzi.

### Co widzisz na wizualizacji? Które drogi mają najwyższe miary centralności? Dlaczego?

## Narysuj histogram miar centralności dla wierzchołków oraz krawędzi
Podpowiedzi:
[https://seaborn.pydata.org/generated/seaborn.histplot.html](https://seaborn.pydata.org/generated/seaborn.histplot.html)
[https://osmnx.readthedocs.io/en/stable/user-reference.html](https://osmnx.readthedocs.io/en/stable/user-reference.html)
[https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.subplots.html](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.subplots.html)

### Jakie własności ma rozkład centralności?

## Wytnij 2% wierzchołków oraz krawędzi o najwyższej mierze centralności. Przedstaw te odcinki na mapie, korzystając z folium, nodes_geojson oraz edges_geojson

Podpowiedź:
[https://pandas.pydata.org/docs/reference/api/pandas.Series.quantile.html](https://pandas.pydata.org/docs/reference/api/pandas.Series.quantile.html)

### Jakie drogi widzisz? Na podstawie np. średniego ruchu z Google Maps czy uważasz, że te drogi mają większy ruch niż inne?

## Na sam koniec zbadamy, jak wypadek samochodowy wpłynie na ruch w Krakowie. W tym celu usuniemy odcinek A4 o nazwie `A4;7`, korzystając z funkcji delete_road. Następnie przypisz miary centralności z pliku do grafu.

In [ ]:
G_no_a4 = ???

with open('no_a4_centrality.pickle', 'rb') as f:
    nodes_cen, edges_cen = pickle.load(f)

## Obliczymy teraz różnicę w centralnościach między normalnym rozkładem Krakowa a scenariuszem, gdzie odcinek A4 jest wyłączony z ruchu. Uzupełnij funkcję, a następnie za jej pomocą oblicz graf rezydualny (różnicę dwóch grafów).

In [25]:
def eval_difference(g1: nx.MultiDiGraph, g2: nx.MultiDiGraph) -> nx.MultiDiGraph:
    g = g1.copy()
    # clean g attrs
    for _, _, a in g.edges(data=True):
        if 'centrality' in a:
            a['centrality'] = -1
    for _, a in g.nodes(data=True):
        if 'centrality' in a:
            a['centrality'] = -1

    # eval diff for edges
    for u, v in g.edges():
        if g1.has_edge(u, v) and g2.has_edge(u, v) and \
           'centrality' in g1[u][v][0] and 'centrality' in g2[u][v][0]:
            g[u][v][0]['centrality'] = ???
    # eval diff for nodes
    for u in g.nodes():
        if g1.has_node(u) and g2.has_node(u) and \
           'centrality' in g1.nodes[u] and 'centrality' in g2.nodes[u]:
            g.nodes[u]['centrality'] = ???

    # drop missing edges between graphs
    edges_to_remove = []
    for u, v, a in g.edges(data=True):
        if a['centrality'] < 0:
            edges_to_remove.append((u, v))
    for u, v in edges_to_remove:
        g.remove_edge(u, v)

    return g

G_diff = ???

## Ponownie sprawdźmy, dla jakich części Krakowa zmieniła się centralność?

In [28]:
nodes, edges = ox.graph_to_gdfs(G_diff)

???

nodes_changed = ???
edges_changed = ???

SyntaxError: invalid syntax (1821817505.py, line 5)

In [33]:
m = folium.Map(location=ox.geocode('Kraków'))
eg = edges_geojson(edges_changed, 'centrality')
ng = nodes_geojson(nodes_changed, 'centrality')
folium.features.GeoJsonPopup(fields=['ref', 'name', 'centrality']).add_to(eg)
folium.features.GeoJsonPopup(fields=['centrality']).add_to(ng)
eg.add_to(m)
ng.add_to(m)
m

/tmp/ipykernel_13745/946581399.py:7: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cmap = cm.get_cmap(cmap)
/tmp/ipykernel_13745/946581399.py:7: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cmap = cm.get_cmap(cmap)


### Dla jakich dróg widzimy zwiększoną centralność?